# 1. Preprocessing

### 1-1. Import Package

In [1]:
import numpy as np

In [45]:
# Data Preprocessing Package
import re
import numpy as np
import pandas as pd
import os
import itertools
import math

# NLP Package
from konlpy.tag import * 
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from collections import Counter

# Visualization Package
import pyLDAvis
import pyLDAvis.gensim_models 
import matplotlib.pyplot as plt
%matplotlib inline

from pprint import pprint
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
 
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [33]:
from konlpy.tag import Mecab 
mecab = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")


In [23]:
mecab.pos('이브라히모비치')

[('이브라히모비치', 'NNP')]

### 1-2. Define Path

In [113]:
"""
해당 셀은 기본적인 경로를 정의하는 셀입니다. 
본인의 환경에 맞춰 경로를 정의하면 됩니다.
"""
# 기본 저장 주소
ROOT_PATH ="./Topic_Modeling/"
# 데이터 저장 주소
DATA_FOLDER_PATH = os.path.join(ROOT_PATH,"1.Data/")
DATA_FILE_NAME = os.path.join(DATA_FOLDER_PATH,"피파_토픽.xlsx") # naver_crawler에서 저장한 데이터 프레임을 입력하세요.
DATA_STOP_WORD_FILE_NAME = os.path.join(DATA_FOLDER_PATH,"Data_stop_word.txt")

# 모델 저장 주소
MODEL_SAVE_FOLDER = os.path.join(ROOT_PATH,"2.Model/")
LDA_MODEL_SAVE_NAME = os.path.join(MODEL_SAVE_FOLDER, "Model_lda_topic_modeling.lda")
OPT_MODEL_SAVE_NAME = os.path.join(MODEL_SAVE_FOLDER, "Model_opt_topic_modeling.lda")
# 결과물 저장 주소
RESULT_FOLDER = os.path.join(ROOT_PATH,"3.Result/")
RESULT_SAVE_LDAVIS = os.path.join(RESULT_FOLDER,"Result_lda_vis.html")
RESULT_TOPIC_EXCEL = os.path.join(RESULT_FOLDER,"Result_topic_excel.xlsx")
RESULT_TOPIC_EXCEL_SUB = os.path.join(RESULT_FOLDER,"Result_topic_excel_weight.xlsx")

In [114]:
DATA_FOLDER_PATH

'./Topic_Modeling/1.Data/'

### 1-3. Load Data

In [117]:
import pickle as pkl
with open(DATA_FOLDER_PATH + 'fifa_data.pkl', 'rb') as f:
    train_ids=pkl.load(f)
    att_masks=pkl.load(f)
    df_fifa=pkl.load(f)
    token_pre=pkl.load(f)

In [8]:
DF_only_text = df_fifa['text_pre']

### 1-4. Tonkinzer, Build Stopword Vocabulary

#### enjeon mecab 설치 블로그 : https://han.gl/I4PrR
#### konlpy mecab 설치 블로그 : https://hong-yp-ml-records.tistory.com/91

In [12]:
stopword_vocab = DATA_STOP_WORD_FILE_NAME # 불용어 파일 불러오기
sep = "\n" # 불용어 처리 인자

print("""
해당 셀은 문서에 대한 토크나이저 및 불용어 처리, corpus를 만드는 셀입니다. 
본 예제에서는 Mecab() 토크나이저를 사용하였으며, 불용어 파일은 1.data 폴더의 메모장에 첨부하였으니 
본인에 맞추어 수정 하시기 바랍니다.
""")
build_corpus=int(input('전체 문서에 대한 워드 카운드 계수를 구하시겠습니까? \n 0 실행 \n 1 미실행 \n'))

def build_vocab(data_frame ,stopword_vocab, separate):
    
    # 불용어 데이터를 가져와 리스트로 변환합니다.
    with open(stopword_vocab, encoding = 'utf-8') as f:
        temp1 = []
        for i in f:
            temp1.append(i)
            
    globals()['stopword_vocab'] = []
    
    # 불용어 데이터는 전역변수 stopword_vocab 선언합니다. 
    # 구분자에 따라 stopword_vocab에 추가하여 불용어 사전을 구축합니다.
    for j in range(len(temp1)):
        temp2 = temp1[j].rstrip(separate)
        globals()['stopword_vocab'].append(temp2)
    
    #okt token에서 명사만 출력합니다. 단어의 길이가 1 초과인 단어만 출력합니다. 
    globals()['list_sent2words'] =[]
    for i in range(len(data_frame)) :
        num_list=[]
        temp = tokenizer.pos(data_frame[i])
        for j in range(len(temp)):
            if temp[j][1] not in ['NNG','NNP','NNB','NR','NP','SL']:
                continue
            if temp[j][0][-1] in ['은','는','이','가']:
                continue

            if len(temp[j][0]) > 1:
                num_list.append(temp[j][0])
        globals()['list_sent2words'].append(num_list)
    
    return [[word for word in doc if word not in globals()['stopword_vocab']] for doc in globals()['list_sent2words']]

result_data =build_vocab(DF_only_text, stopword_vocab, sep)


# 전체 기사에 대한 워드 카운트 계수 확인

def word_corpus(result_data):
    #전체 단어의 갯수 파악
    words = list(itertools.chain(*result_data))
    print('전체 워드의 개수 : {}'.format(len(words)))

    #단어의 빈도수를 확인 후 추가할 불용어 확인 작업
    vocab = Counter(words)
    vocab_size = len(words)
    vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 n개의 단어만 저장 vocab
    return vocab

vocab=word_corpus(result_data)

if build_corpus==0:
    df_corpus=pd.DataFrame(columns=["text","count"])
    tmp_list=[]
    tmp_list1=[]
    for word, num in vocab:
        tmp_list.append(word)
        tmp_list1.append(num)
    df_corpus['text']=tmp_list
    df_corpus['count']=tmp_list1
    #상위 20개의 워드 카운드 계수만 출력
    print(df_corpus.head(20))



해당 셀은 문서에 대한 토크나이저 및 불용어 처리, corpus를 만드는 셀입니다. 
본 예제에서는 OKT() 토크나이저를 사용하였으며, 불용어 파일은 1.data 폴더의 메모장에 첨부하였으니 
본인에 맞추어 수정 하시기 바랍니다.

전체 문서에 대한 워드 카운드 계수를 구하시겠습니까? 
 0 실행 
 1 미실행 
0
전체 워드의 개수 : 253576
   text  count
0    게임   4647
1    선수   4240
2    넥슨   2550
3    유저   2545
4    피파   2416
5    보정   1891
6    시즌   1808
7    사람   1713
8    정도   1514
9    패스   1499
10   문제   1460
11   현질   1208
12   체감   1153
13   급여   1143
14   추천   1120
15   강화   1089
16  스쿼드   1086
17   보상    923
18   축구    841
19   패치    838


# 2. Build Architecture

### 2-1. IDs, Words and Documents Mapping 

In [13]:
"""
해당 셀은 단어들을 ID와 Mapping한 뒤, 단어들(정확히는 단어들의 ID)과 문서를 Mapping하는 셀입니다.
"""

# 토픽 모델링 딕셔너리 생성
id2word = corpora.Dictionary(result_data)
 
# 토픽모델링에 사용할 말뭉치 생성
texts = result_data
 
# 용어-문서 빈도
corpus = [id2word.doc2bow(text) for text in texts]


### 2-2. Define Hyperparameter and modeling

#### 토픽 모델링 평가 참고용 블로그 : https://bab2min.tistory.com/587
#### Perplexity :  혼란도 -> 작으면 작을수록 실제 문헌의 결과를 잘 반영( 학습이 잘 되었다.)
#### Coherence : 일관성 -> 토픽이 얼마나 의미론적으로 일관성이 있는지, 높을수록 일관성이 높다(학습이 잘 되었다)

In [72]:
"""
해당 셀은 토픽모델링(LDA)를 학습시키기 위한 하이퍼파라미터를 사전에 정의하는 셀입니다.
하이퍼파라미터에 대한 설명은 다음 링크에서 확인하십시오.
https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html#training
"""

print('토픽 기본 모델링을 실시 합니다. 해당 모델은 "lda_model" 변수로 입력됩니다.')
print(' ')

NUM_TOPICS = int(input('토픽의 개수를 입력해 주세요. '))
TOPICS_W_NUM = int(input('출력할 토픽별 단어의 개수를 입력해 주세요. '))
save_lda_model= int(input("선택한 토픽 모델을 저장하시겠습니까? \n0 저장  \n1 미저장  "))

RANDOM_STATE = 100
UPDATE_EVERY = 1
CHUNKSIZE = 100
PASSES = 10
ALPHA = 'auto'
PER_WORD_TOPICS = True

#해당 셀은 토픽모델링(LDA)에 대해 모델을 정의하는 셀입니다.
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, 
                                            num_topics=NUM_TOPICS, random_state=RANDOM_STATE, 
                                            update_every=UPDATE_EVERY, chunksize=CHUNKSIZE,
                                            passes=PASSES, alpha=ALPHA, per_word_topics=PER_WORD_TOPICS)

# 토픽 출력
pprint(lda_model.print_topics(num_words=TOPICS_W_NUM))
doc_lda = lda_model[corpus]

"""
해당 셀은 설계한 모델을 계산하는 셀입니다.
측정은 Perplexity와 Coherence Score입니다.
"""

# Perplexity 
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.
 
# Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=result_data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# 모델 저장 
if save_lda_model == 0:
    lda_model.save(LDA_MODEL_SAVE_NAME)


토픽 기본 모델링을 실시 합니다. 해당 모델은 "lda_model" 변수로 입력됩니다.
 
토픽의 개수를 입력해 주세요. 10
출력할 토픽별 단어의 개수를 입력해 주세요. 20
선택한 토픽 모델을 저장하시겠습니까? 
0 저장  
1 미저장  0
[(0,
  '0.065*"가요" + 0.062*"감사" + 0.035*"첼시" + 0.033*"박정무" + 0.027*"점검" + '
  '0.026*"접속" + 0.023*"포메이션" + 0.021*"피시방" + 0.018*"2억" + 0.017*"드록바" + '
  '0.017*"반데이크" + 0.016*"하향" + 0.015*"감독모드" + 0.014*"최소" + 0.014*"클래스" + '
  '0.013*"동일" + 0.012*"터짐" + 0.012*"이놈" + 0.011*"실장" + 0.011*"연장"'),
 (1,
  '0.079*"선수" + 0.047*"시즌" + 0.037*"보정" + 0.033*"정도" + 0.019*"문제" + 0.013*"상대" '
  '+ 0.012*"시세" + 0.011*"패치" + 0.011*"경기" + 0.011*"출시" + 0.010*"운영" + '
  '0.010*"상황" + 0.010*"사용" + 0.010*"시스템" + 0.010*"구단가치" + 0.009*"체감" + '
  '0.009*"가치" + 0.009*"구단" + 0.008*"능력치" + 0.008*"답변"'),
 (2,
  '0.031*"티어" + 0.027*"19" + 0.025*"댓글" + 0.023*"스텟" + 0.021*"결과" + 0.021*"서버" '
  '+ 0.019*"호날두" + 0.018*"고민" + 0.017*"신규" + 0.015*"10억" + 0.015*"나머지" + '
  '0.014*"최고" + 0.013*"4카" + 0.013*"성능" + 0.013*"토트넘" + 0.013*"국대" + '
  '0.012*"선택" + 0.012*"지단" + 0.011*"판매" + 0.011*"

In [89]:
topic_words=lda_model.show_topics(num_words=10,formatted=False)

In [107]:
for i in range(0, 10):
    for j in range(0,10):
        print(topic_words[i][1][j][0])
    print("")

가요
감사
첼시
박정무
점검
접속
포메이션
피시방
2억
드록바

선수
시즌
보정
정도
문제
상대
시세
패치
경기
출시

티어
19
댓글
스텟
결과
서버
호날두
고민
신규
10억

가격
아이콘
가능
20토티
금카
토티
은카
오버롤
기준
19토티

보상
패키지
스쿼드
이벤트
업데이트
카드
귀속
버닝
아이폰
프로

부탁
드립니다
피파온라인4
수수료
의견
친구
골키퍼
호나우두
7카
검색

급여
추천
강화
피파4
버그
확률
5카
3카
구매
월클

게임
넥슨
피파
유저
현질
사람
시작
적폐
무과금
확인

매물
챔스
전술
20
과금
방법
참고
날두
맨유
레알

크로스
패스
중거리
축구
굴리트
키퍼
에이전트
개인
적용
메시



In [111]:
"""
위의 셀에서 학습한 모델을 시각화 하여 HTML 파일로 저장하는 셀입니다. 
문서의 양이 많을 경우 오래 걸리니 참고 바랍니다. 
"""
def create_vis(model):
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim_models.prepare(model, corpus, id2word, sort_topics=False)
    pyLDAvis.save_html(vis, RESULT_SAVE_LDAVIS)
    return vis

In [112]:
create_vis(lda_model)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.082121  0.044092       1        1   2.759657
1      0.429074  0.077692       2        1  25.701308
2     -0.069977  0.047875       3        1   7.224766
3     -0.073308  0.048466       4        1   5.176474
4     -0.038334  0.034267       5        1  10.462414
5     -0.074490  0.048151       6        1   6.566399
6      0.016709  0.032045       7        1  12.709849
7      0.028914 -0.422809       8        1  14.565076
8     -0.080777  0.047733       9        1   5.029229
9     -0.055690  0.042489      10        1   9.804828, topic_info=     Term         Freq        Total Category  logprob  loglift
35     게임  4094.000000  4094.000000  Default  30.0000  30.0000
130    선수  5171.000000  5171.000000  Default  29.0000  29.0000
187    넥슨  3211.000000  3211.000000  Default  28.0000  28.0000
49     피파  3104.000000  3104.000000  Default  27.0000  27.0000
166    유저  2760.000000  2760.000000  Default  26.0000  26.0000
...   ...          ...          ...      ...      ...      ...
1686   복귀   162.158455   163.122804  Topic10  -5.0325   2.3164
397    언급   155.812985   156.777412  Topic10  -5.0725   2.3161
3418   감독   145.082779   146.047057  Topic10  -5.1438   2.3157
388   개돼지   143.555370   144.519688  Topic10  -5.1544   2.3156
1446   원톱   142.607860   143.572244  Topic10  -5.1610   2.3156

[332 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
280       1  0.984448  100만원
2294      3  0.997632    10억
1397      6  0.995091    10판
1662      3  0.998564     19
8343      4  0.994551   19토티
...     ...       ...    ...
314       3  0.998261    호날두
723       7  0.998621     확률
169       8  0.998253     확인
3507      1  0.980433     효과
2975      9  0.994100     후기

[306 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [108]:
# 위에서 선택한 토픽 모델을 엑셀 파일로 떨어뜨리기 위한 함수 
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=result_data):
    sent_topics_df = pd.DataFrame()
    for i, row in enumerate(ldamodel[corpus]):
        topics_info_by_doc = row[0]
        topics_info_by_doc = sorted(topics_info_by_doc, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(topics_info_by_doc):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic, 4), topic_keywords]),
                                                       ignore_index=True)
            else:
                break

    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [110]:
'''
확인하고자 하는 모델을 입력하여 데이터 프레임으로 만든 후, 최종 엑셀파일을 저장합니다.
lda_model 또는 optimal_model 모델을 입력해주시기 바랍니다.
'''
model_name = int(input('모델을 선택하세요. \n0 lda_model \n1 optimal_model  '))
if model_name == 0:
    df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=result_data)
    topic_weight=lda_model[corpus]
    df_topic_weight = pd.DataFrame()
    for i in range(0, lda_model.num_topics):
        df_topic_weight['topic{}'.format(i)]=pd.Series()

if model_name ==1:
    df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=result_data)
    topic_weight=optimal_model[corpus]
    df_topic_weight = pd.DataFrame()
    for i in range(0, optimal_model.num_topics):
        df_topic_weight['topic{}'.format(i)]=pd.Series()

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
 

# LDA모델에서 토픽 웨이트를 추출하여 데이터 프레임에 저장 
for num, topic in enumerate(topic_weight):
    for i in topic[0]:
        df_topic_weight.loc[num, 'topic' + str(i[0])] = i[1]

# Null값은 아주 작은 값으로 대체
df_topic_weight = df_topic_weight.fillna(math.exp(-1000)) 

#binary 생성
df_topic_binary = pd.DataFrame()
for i in range(0, lda_model.num_topics):
    df_topic_binary['topic{}'.format(i)]=df_topic_weight["topic{}".format(i)].apply(lambda x: 1 if x>0 else x)


#데이터 프레임 연결
df_topic_last = pd.concat([df_fifa, df_dominant_topic, df_topic_binary], axis=1)
print(df_topic_last)
df_topic_last_w = pd.concat([df_fifa, df_dominant_topic, df_topic_weight], axis=1)
# 마지막 엑셀 저장
df_topic_last_w.to_excel(RESULT_TOPIC_EXCEL_SUB, sheet_name = "sheet1")

모델을 선택하세요. 
0 lda_model 
1 optimal_model  0
                       day                          title  \
0      2021-09-11 00:00:00                5백 2볼란치 는 비매너다?   
1      2021-09-11 00:00:00                         패키지 추천   
2      2021-09-11 00:00:00          20A 메시 로패하면 오버롤 오르나요?   
3      2021-09-11 00:00:00                             알파   
4      2021-09-11 00:00:00                 ebs 로랑 코시엘니 은카   
...                    ...                            ...   
17227  2020-01-01 00:00:00                           강화확률   
17228  2020-01-01 00:00:00                지금 토티 시즌 사도 될까요   
17229  2020-01-01 00:00:00            토트넘 티비금카팀 선수고민중 입니다   
17230  2020-01-01 00:00:00               아래 ICON 개봉 결과다..   
17231  2020-01-01 00:00:00  이번 9900개 패키지사면 호갱이들 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ   

                                                 content   view  hit  \
0      친선에서 이런 채팅을 하시는 분이 있어서 한번 이야기 해보고 싶네요.해당 경기 내용...    161    0   
1      9000fc로추석 풍년 + 별토끼 3개추석 연쇄 패키지 2개중에 구매하려는데 어떤게...     79    0   
2      